<H1> Notebook to verify the calculations of our simulator </H1>

## Importing required libraries

In [1]:
# importaing standard libraries
import zipfile
import pickle
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.signal import freqs,periodogram,cheby1
import numpy as np

In [2]:
# import quantum libraries
import qutip

from qmldataset import run_default_experiment

2021-09-05 21:26:08.202287: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-05 21:26:08.202336: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Create a set of data
The package uses numerical libraries to simulate the quantum system, the control and noise signals (of predefined types). In this notebook, we will use one of the default 1-qubit experiments - `1q_XZ_N4`.

In [ ]:
# first, let us create a dataset, if it does not exist
# other choices for defualt experiments in the pacakge are
# ['1q_X', '1q_XZ_N1', '1q_XZ_N2', '1q_XZ_N3', '1q_XZ_N4',
#  '1q_XY', '1q_XY_XZ_N1N5', '1q_XY_XZ_N1N6', '1q_XY_XZ_N3N6',
#  '2q_IX_XI_XX', '2q_IX_XI_XX_IZ_ZI_N1N5', '2q_IX_XI_XX_IZ_ZI_N1N6',
#  '2q_IX_XI_IZ_ZI_N1N6']

experiment = '1q_XZ_N4'

num_examples = 10 # other options, 10000
batch_size = 5     # other options, 50
output_location = "/home/rchakrav/progs/qmldataset_result/"
pulse_shape = "Square"

run_default_experiment(
    experiment_config=experiment,
    pulse_shape=pulse_shape,
    num_examples=num_examples,
    batch_size=batch_size,
    output_location=output_location
)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
signal_generator_6 (SignalGener ((None, 5, 3), (None 0           input_7[0][0]                    
__________________________________________________________________________________________________
H0 (HamiltonianConstruction)    (None, 1024, 1, 2, 2 0           signal_generator_6[0][1]         
__________________________________________________________________________________________________
noise_layer_6 (NoiseLayer)      (None, 1024, 2000, 1 0           input_7[0][0]                    
____________________________________________________________________________________________

1/1 [==============================] - 9s 9s/step
Analyzing results

Measurement are:
[[-0.7758454  -0.03551561  0.51069826  0.77583265  0.03552258 -0.5106849
   0.45605248 -0.22237691  0.8013504  -0.45606455  0.22238392 -0.80133575
   0.07962628  0.89062184  0.20841704 -0.07963863 -0.89061505 -0.20840317]]
The Vo operators are:
[array([[[ 0.85371244-0.23721202j,  0.17671648+0.j        ],
        [-0.17672428+0.j        ,  0.85371244+0.23721202j]]],
      dtype=complex64), array([[[0.88592553-0.24314377j, 0.        +0.00204751j],
        [0.        +0.00205569j, 0.88592553+0.24314377j]]],
      dtype=complex64), array([[[ 0.95693254+0.j        ,  0.16494134+0.05959502j],
        [-0.16494134+0.05959502j,  0.9569232 +0.j        ]]],
      dtype=complex64)]
The distance measures are:
[0.33000342595877835, 0.26858570518399455, 0.18059377416498923]
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)               

In [5]:
# verification parameters
dataset = "S_1q_XZ_N3_distortion"
ex_num  = 1

In [8]:
fname = "%s_ex_%d"%(dataset, ex_num)

# unzip the dataset zipfile and extract the example file
fzip  = zipfile.ZipFile(os.path.join(output_location, "%s.zip"%dataset), mode='r')
fzip.extract( fname )
fzip.close()

# load the example file
f     = open(fname,  "rb")
data  = pickle.load(f)
f.close()
os.remove(fname)

KeyError: "There is no item named 'S_1q_XZ_N3_distortion_ex_1' in the archive"

In [ ]:
# plot the pulse
plt.figure()
num_controls = len(data["sim_parameters"]["dynamic_operators"])
for idx in range(num_controls):
    plt.subplot(num_controls , 1, idx+1 )
    plt.plot(data["time_range"], data["pulses"][0,:,0,idx], label="undistorted")
    plt.plot(data["time_range"], data["distorted_pulses"][0,:,0,idx], label="distorted")
    plt.xlabel('t')
    plt.ylabel('f(t)')
    plt.grid()
    plt.legend()
print(data["pulse_parameters"])

In [ ]:
# display distortion filter if exists
distortion = cheby1(4,0.1,2*np.pi*20, analog=True)
# evaluate frequency response of the filter
w, Hw = freqs(distortion[0], distortion[1])
plt.figure(figsize=[15,4])
plt.subplot(1,2,1)
plt.semilogx(w, 20*np.log(np.abs(Hw)))
plt.xlabel(r'$\Omega$')
plt.ylabel(r'$|H(\Omega)|$')
plt.grid()
plt.subplot(1,2,2)
plt.semilogx(w, np.angle(Hw))
plt.xlabel(r'$\Omega$')
plt.ylabel(r'arg $H(\Omega)$')
plt.grid()

In [ ]:
# display noise if exists
for idx_profile,profile in enumerate(data["sim_parameters"]["noise_profile"]): 
    if profile in [2,3,4] or (profile==6 and p==0): 
        # estimate the correlation matrix of the noise
        correlation = 0
        for k in range(data["sim_parameters"]["K"]):
            correlation = correlation + data["noise"][0,:,k:k+1,idx_profile]@data["noise"][0,:,k:k+1,idx_profile].T
        correlation = correlation/data["sim_parameters"]["K"]
        # plot correlation matrix
        plt.figure()
        plt.matshow(correlation,0)
        plt.colorbar()
        p = 0
    elif profile in [1,5]:
        # estimate the PSD of the noise
        psd = 0
        for k in range(data["sim_parameters"]["K"]):
            f, Pxx = periodogram(data["noise"][0,:,k,idx_profile], data["sim_parameters"]["M"]/data["sim_parameters"]["T"])            
            psd = psd + Pxx
        psd = psd/data["sim_parameters"]["K"]
        plt.figure()
        plt.plot(f[f>0], psd[1:])
        plt.xlabel('f')
        plt.ylabel('psd')
        plt.grid()
        p = 1

In [ ]:
# load initial states, measurement operators, and control Hamilotonian
initial_states = [qutip.Qobj(state) for state in data["sim_parameters"]["initial_states"] ] 
measurements   = [qutip.Qobj(op) for op in data["sim_parameters"]["measurement_operators"] ]

H0  = [ [qutip.Qobj(op), np.ones((len(data["time_range"])))] for op in data["sim_parameters"]["static_operators"] ] + [ [qutip.Qobj(op), data["distorted_pulses"][0,:,0,idx]] for idx, op in enumerate(data["sim_parameters"]["dynamic_operators"]) ]

expectations = np.zeros((1,data["sim_parameters"]["K"], len(initial_states)*len(measurements)))  
for idx_K in range(data["sim_parameters"]["K"]):    
    H1      = [ [qutip.Qobj(op), data["noise"][0,:,idx_K,idx]] for idx, op in enumerate(data["sim_parameters"]["noise_operators"]) ]
    results = [ qutip.mesolve(H0 + H1, rho, np.array(data["time_range"]), e_ops=measurements).expect for rho in initial_states]     
    expectations [0,idx_K, :] = np.concatenate( [np.array( [results[idx_rho][idx_M][-1] for idx_M in range(len(measurements))]) for idx_rho in range(len(initial_states))])
    print(idx_K+1,  end="\r")

In [ ]:
# plot the average expectation over all noise realizations for every observable
plt.figure()
plt.plot(np.average(expectations, 1)[0], label="qutip")
plt.plot(data["expectations"][0], label = "tf")
plt.ylabel("Average observable value")
plt.xlabel("observable Index")
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.legend()
plt.grid()

In [ ]:
# plot all possible observables for a particular noise realization
idx_K = 2
plt.figure()
plt.plot(expectations[0,idx_K,:], label="qutip")
plt.plot(data["Eo"][0,idx_K,:],  label = "tf")
plt.ylabel("Observable Value for realization %d"%idx_K)
plt.xlabel("Observable Index")
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
plt.legend()
plt.grid()

In [ ]:
# simulation time without I/O operations
print("Total time for 1 batch is %f seconds"%data["sim_parameters"]["elapsed_time"])
print("Average time for 1 example %f seconds"%(data["sim_parameters"]["elapsed_time"]/ data["sim_parameters"]["batch_size"]))